In [6]:
import numpy as np
import pandas as pd
import requests
import json

In [14]:
url = "../../../datasets/Iowa_Liquor_Sales.csv"
alc_df = pd.read_csv(url, nrows=10000)
print("test")
print(alc_df.head(1))

test
  Invoice/Item Number        Date  Store Number  \
0     INV-09962700007  01/23/2018          4310   

                       Store Name            Address      City  Zip Code  \
0  Fareway Stores #502 / Cherokee  205 West Bluff St  Cherokee   51012.0   

                Store Location  County Number    County  ...  Item Number  \
0  POINT (-95.551866 42.75508)           18.0  CHEROKEE  ...        14192   

            Item Description  Pack Bottle Volume (ml)  State Bottle Cost  \
0  Pendleton Canadian Whisky    12                750               13.1   

  State Bottle Retail  Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
0               19.65             3           58.95                  2.25   

   Volume Sold (Gallons)  
0                   0.59  

[1 rows x 24 columns]


In [21]:
def return_name(string):
  return " ".join(string.split(" ")[:-1])

pop_df= pd.read_csv("https://data.iowa.gov/resource/qtnr-zsrc.csv")
pop_df["county"] = pop_df["geographicname"].apply(return_name).str.upper()
print(pop_df.head(1))


    fips geographicname                     year  population  \
0  19169   Story County  2011-07-01T00:00:00.000       91136   

                    primary_point county  
0  POINT (-93.4650448 42.0362415)  STORY  


In [19]:
def return_year(string):
  return string.split("-")[0]

pop_df["year"] = pop_df["year"].apply(return_year)


In [22]:
alc_df["Year"] = alc_df["Date"].apply(return_year)
alc_df["County"] = alc_df["County"].str.upper()


In [23]:

alc_df["County"].unique()

array(['CHEROKEE', 'BLACK HAWK', 'MARSHALL', 'CLAYTON', 'STORY',
       'JOHNSON', 'LINN', 'POLK', 'PLYMOUTH', 'LUCAS', 'DUBUQUE',
       'CLINTON', 'SCOTT', 'PALO ALTO', 'DALLAS', 'WEBSTER', 'BOONE',
       'TAMA', 'BREMER', 'CERRO GORD', 'POTTAWATTA', 'MAHASKA',
       'WINNESHIEK', 'KOSSUTH', 'MUSCATINE', 'LYON', 'WOODBURY', 'MONONA',
       'DICKINSON', 'HARRISON', 'MILLS', 'CRAWFORD', 'OBRIEN',
       'DES MOINES', 'WASHINGTON', 'APPANOOSE', 'BUTLER', 'WARREN',
       'CLAY', nan, 'GRUNDY', 'CARROLL', 'SIOUX', 'BENTON', 'OSCEOLA',
       'ALLAMAKEE', 'FRANKLIN', 'EMMET', 'WAPELLO', 'MARION',
       'POCAHONTAS', 'HUMBOLDT', 'IOWA', 'MADISON', 'LEE', 'DAVIS',
       'BUCHANAN', 'BUENA VIST', 'WRIGHT', 'JACKSON', 'IDA', 'JONES',
       'FLOYD', 'POWESHIEK', 'CLARKE', 'WAYNE', 'UNION', 'SAC', 'CALHOUN',
       'CASS', 'WORTH', 'JASPER', 'PAGE', 'HANCOCK', 'GUTHRIE',
       'MONTGOMERY', 'DELAWARE', 'LOUISA', 'HARDIN', 'JEFFERSON',
       'FAYETTE', 'MITCHELL', 'TAYLOR', 'CHICKASAW', 

In [24]:
pop_df["county"].unique()

array(['STORY', 'FAYETTE', 'HOWARD', 'APPANOOSE', 'CLAYTON', 'JOHNSON',
       'MONONA', 'SIOUX', 'JEFFERSON', 'LINN', 'RINGGOLD', 'POCAHONTAS',
       'POLK', 'HANCOCK', 'WEBSTER', 'TAMA', 'CEDAR', "O'BRIEN", 'EMMET',
       'GUTHRIE', 'PALO ALTO', 'CHICKASAW', 'OSCEOLA', 'MONTGOMERY',
       'BLACK HAWK', 'WASHINGTON', 'LYON', 'LUCAS', 'DICKINSON', 'MILLS',
       'BENTON', 'DUBUQUE', 'JONES', 'PAGE', 'SCOTT', 'WAYNE',
       'VAN BUREN', 'CRAWFORD', 'POTTAWATTAMIE', 'LEE', 'CHEROKEE',
       'TAYLOR', 'GREENE', 'MAHASKA', 'HARRISON', 'HUMBOLDT',
       'CERRO GORDO', 'CALHOUN', 'UNION', 'WINNEBAGO', 'MITCHELL',
       'DAVIS', 'DALLAS', 'KOSSUTH', 'JACKSON', 'ALLAMAKEE', 'BUTLER',
       'MUSCATINE', 'IOWA', 'ADAIR', 'DELAWARE', 'FLOYD', 'WARREN',
       'WORTH', 'AUDUBON', 'MADISON', 'CLAY', 'BREMER', 'DECATUR',
       'LOUISA', 'GRUNDY', 'WOODBURY', 'FRANKLIN', 'BUENA VISTA',
       'CARROLL', 'KEOKUK', 'CLARKE', 'SHELBY', 'SAC', 'JASPER',
       'POWESHIEK', 'HENRY', 'CLINTON', '

In [25]:
df = pd.read_html("https://www.iowa-demographics.com/counties_by_population")[0]
df["year"] = "2021"
df["county"] = df["County"].apply(return_name).str.upper()
df["population"] = df["Population"]
df = df[["county", "population", "year"]]
df

,county,population,year
0,POLK,496844,2021
1,LINN,228939,2021
2,SCOTT,174170,2021
3,JOHNSON,154748,2021
4,BLACK HAWK,130368,2021
...,...,...,...
95,TAYLOR,5868,2021
96,AUDUBON,5635,2021
97,RINGGOLD,4639,2021
98,ADAMS,3641,2021


In [26]:
pop_df = pop_df[["county", "population", "year"]]
pop_df = pd.concat([pop_df, df])

In [27]:
combined = alc_df.merge(pop_df, how = "left")
combined[combined["year"].equals(combined["Year"])]

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False